In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

merged = pd.read_csv('merged.csv')

ModuleNotFoundError: No module named 'pandas'

In [ ]:
import pandas as pd
import numpy as np


# Group the data by Inflation and by date
a = merged.copy()
a = a[a['date']>'2012']
grouped = a.groupby(['Inflation', 'date'])
total_returns = []

# Iterate through the grouped data and select the top 10 and bottom 10 industries by return
for name, group in grouped:
    inflation, date = name
    sorted_group = group.sort_values("MthRet", ascending=False)
    top_10 = sorted_group.head(10)
    bottom_10 = sorted_group.tail(10)
    top_10_return = np.mean(top_10["MthRet"])
    bottom_10_return = np.mean(bottom_10["MthRet"])
    total_return = top_10_return - bottom_10_return
    total_returns.append(total_return)
    #Ignore this, this was just so I could check some of the stuff
    #print("Date: {}, Inflation: {}\nTop 10 Industries by Return: {}\nBottom 10 Industries by Return: {}\nTotal Return: {}\n".format(date, inflation, top_10_return, bottom_10_return, total_return))

total_returns = [x for x in total_returns if not np.isnan(x)]
average_total_return = np.mean(total_returns)

average_total_return



1.5118729190476188

In [ ]:
import pandas as pd


# Group the data by date and get the monthly return for each industry
grouped_by_date = a.groupby("MthCalDt")

# Create an empty list to store the weighted monthly return for the top 10% of industries for each date
monthly_return_top_10_percent_weighted = []

# Iterate through each date group
for date, date_group in grouped_by_date:
    # Sort the date group by monthly return
    date_group_sorted = date_group.sort_values("MthRet", ascending=False)
    
    # Get the number of industries for this date group
    n = len(date_group_sorted)
    
    # Get the top 10% of industries
    top_10_percent = date_group_sorted.iloc[:int(n * 0.1)]
    
    # Calculate the weighted monthly return for the top 10% of industries
    weighted_monthly_return = (top_10_percent["MthRet"] * top_10_percent["ShrOut"]).sum() / top_10_percent["ShrOut"].sum()
    
    # Append the result to the list
    monthly_return_top_10_percent_weighted.append((date, weighted_monthly_return))

# Convert the list to a dataframe
monthly_return_top_10_percent_weighted_df = pd.DataFrame(monthly_return_top_10_percent_weighted, columns=["MthCalDt", "Weighted_Avg_Monthly_Return"])


In [ ]:
Industry_Inflation_group

Industry Title                                    Inflation
BIOLOGICAL PRODUCTS, (NO DISGNOSTIC SUBSTANCES)   deflation   -0.000049
                                                  low          0.005287
                                                  mid          0.017790
                                                  high         0.030621
BOTTLED & CANNED SOFT DRINKS & CARBONATED WATERS  deflation    0.012558
                                                                 ...   
TELEPHONE COMMUNICATIONS (NO RADIOTELEPHONE)      high         0.012059
TRUCKING (NO LOCAL)                               deflation    0.019999
                                                  low          0.008438
                                                  mid          0.014196
                                                  high         0.004208
Name: MthRet, Length: 232, dtype: float64

In [ ]:
import pandas as pd
import numpy as np


# Group the data by Inflation and by date
a = merged.copy()
a = a[a['date']>'2012']
grouped = a.groupby(['Inflation', 'date'])
total_returns = []

# Iterate through the grouped data and select the top 10 and bottom 10 industries by return
for name, group in grouped:
    inflation, date = name
    sorted_group = group.sort_values("MthRet", ascending=False)
    top_10 = sorted_group.head(10)
    bottom_10 = sorted_group.tail(10)
    top_10_return = np.mean(top_10["MthRet"])
    bottom_10_return = np.mean(bottom_10["MthRet"])
    total_return = top_10_return - bottom_10_return
    total_returns.append(total_return)
    #Ignore this, this was just so I could check some of the stuff
    #print("Date: {}, Inflation: {}\nTop 10 Industries by Return: {}\nBottom 10 Industries by Return: {}\nTotal Return: {}\n".format(date, inflation, top_10_return, bottom_10_return, total_return))

total_returns = [x for x in total_returns if not np.isnan(x)]
average_total_return = np.mean(total_returns)

average_total_return



1.5118729190476188

In [ ]:
import pandas as pd
import numpy as np


# Group the data by Inflation and by date
a = merged.copy()
a = a[a['date']>'1947']
grouped = a.groupby(['Inflation', 'date'])
total_returns = []

# Calculate the top 10 and bottom 10 industries for each inflation category over the entire duration
top_industries = {}
bottom_industries = {}
for inflation in a['Inflation'].unique():
    if pd.isna(inflation):
        continue
    inflation_group = a[a['Inflation']==inflation]
    sorted_group = inflation_group.sort_values("MthRet", ascending=False)
    top_10 = list(sorted_group.head(1000)['Industry Title'].unique())
    bottom_10 = list(sorted_group[sorted_group["MthRet"] < 0]['Industry Title'].unique()[:1000])
    top_industries[inflation] = top_10
    bottom_industries[inflation] = bottom_10

# Iterate through the grouped data and calculate the average return for each date
for name, group in grouped:
    date = name[1]
    inflation = group['Inflation'].iloc[0]
    if pd.isna(inflation):
        continue
    top_10_industries = top_industries[inflation]
    bottom_10_industries = bottom_industries[inflation]
    top_10 = group[group['Industry Title'].isin(top_10_industries)]
    bottom_10 = group[group['Industry Title'].isin(bottom_10_industries)]
    top_10_return = np.mean(top_10["MthRet"])
    bottom_10_return = np.mean(bottom_10["MthRet"])
    total_return = top_10_return - bottom_10_return
    total_returns.append(total_return)


total_returns = [x for x in total_returns if not np.isnan(x)]
average_total_return = np.mean(total_returns)

average_total_return


0.0001315082076721943

In [ ]:
#You have to run this in collab
import pandas as pd
import numpy as np


def find_last_n_inflation_periods(inflation, date, n):
    """
    Finds the last n periods with the same inflation status as the current date.
    """
    a = merged.copy()
    a = a[a['date'] > '1947']
    a = a[a['Inflation'] == inflation]
    a = a[a['date'] < date]
    a = a.sort_values('date', ascending=False)
    last_n_periods = a['date'].iloc[:n].tolist()
    return last_n_periods


grouped = merged.groupby(['Inflation', 'date'])
total_returns = []

# Calculate the top 10 and bottom 10 industries for each date
for name, group in grouped:
    date = name[1]
    inflation = group['Inflation'].iloc[0]
    if pd.isna(inflation):
        continue
    last_n_periods = find_last_n_inflation_periods(inflation, date, 5)
    if not last_n_periods:
        continue
    inflation_group = merged[merged['date'].isin(last_n_periods) & (merged['Inflation'] == inflation)]
    sorted_group = inflation_group.sort_values("MthRet", ascending=False)
    top_10 = list(sorted_group.head(10)['Industry Title'].unique())
    bottom_10 = list(sorted_group[sorted_group["MthRet"] < 0]['Industry Title'].unique()[:10])
    top_10_industries = group[group['Industry Title'].isin(top_10)]
    bottom_10_industries = group[group['Industry Title'].isin(bottom_10)]
    top_10_return = np.mean(top_10_industries["MthRet"])
    bottom_10_return = np.mean(bottom_10_industries["MthRet"])
    total_return = top_10_return - bottom_10_return
    total_returns.append(total_return)

total_returns = [x for x in total_returns if not np.isnan(x)]
average_total_return = np.mean(total_returns)

average_total_return


KeyboardInterrupt: 

In [ ]:
total_returns = [x for x in total_returns if not np.isnan(x)]


In [ ]:
np.mean(total_returns)

0.002202447710998025

In [ ]:
df = a.copy()
# First, we need to calculate the market capitalization of each company in the dataset
df['MthPrc'] = df['MthPrc'].str.replace(',','')
df['ShrOut'] = df['ShrOut'].astype(float)
df['MthPrc'] = df['MthPrc'].astype(float)
df['MarketCap'] = df['ShrOut'] * df['MthPrc']

# Next, we'll group the data by date and calculate the total market capitalization for each date
grouped = df.groupby('date')
total_market_cap = grouped['MarketCap'].sum()

# Now we'll calculate the market weight of each company on each date by dividing its market capitalization by the total market capitalization
df['MarketWeight'] = df.apply(lambda x: x['MarketCap']/total_market_cap[x['date']], axis=1)

# Finally, we'll multiply the return of each company by its market weight and sum up the results to get the market weighted value portfolio return for each date
df = df.dropna(subset=['MarketWeight', 'MthRet'])
df['MarketWeightedReturn'] = df['MarketWeight'] * df['MthRet']

market_weighted_return = df.groupby('date')['MarketWeightedReturn'].sum()

market_weighted_return.mean()


C:\Users\prana\AppData\Local\Temp\ipykernel_21912\3893374852.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0.015863126089658244

0.015863126089658244

In [ ]:
import pandas as pd

# Group the data by date and get the monthly return for each industry
grouped_by_date = a.groupby("MthCalDt")

# Create an empty list to store the monthly return for the top 10% of industries for each date
monthly_return_top_10_percent = []

# Iterate through each date group
for date, date_group in grouped_by_date:
    # Sort the date group by monthly return
    date_group_sorted = date_group.sort_values("MthRet", ascending=False)
    
    # Get the number of industries for this date group
    n = len(date_group_sorted)
    
    # Get the top 10% of industries
    top_10_percent = date_group_sorted.iloc[:int(n * 0.1)]
    
    # Calculate the average monthly return for the top 10% of industries
    avg_monthly_return = top_10_percent["MthRet"].mean()
    
    # Append the result to the list
    monthly_return_top_10_percent.append((date, avg_monthly_return))

# Convert the list to a dataframe
monthly_return_top_10_percent_df = pd.DataFrame(monthly_return_top_10_percent, columns=["MthCalDt", "Avg_Monthly_Return"])


In [ ]:
monthly_return_top_10_percent_df['Avg_Monthly_Return'].mean()

0.2650357963888545

In [ ]:
import pandas as pd


# Group the data by date and get the monthly return for each industry
grouped_by_date = a.groupby("MthCalDt")

# Create an empty list to store the weighted monthly return for the top 10% of industries for each date
monthly_return_top_10_percent_weighted = []

# Iterate through each date group
for date, date_group in grouped_by_date:
    # Sort the date group by monthly return
    date_group_sorted = date_group.sort_values("MthRet", ascending=False)
    
    # Get the number of industries for this date group
    n = len(date_group_sorted)
    
    # Get the top 10% of industries
    top_10_percent = date_group_sorted.iloc[:int(n * 0.1)]
    
    # Calculate the weighted monthly return for the top 10% of industries
    weighted_monthly_return = (top_10_percent["MthRet"] * top_10_percent["ShrOut"]).sum() / top_10_percent["ShrOut"].sum()
    
    # Append the result to the list
    monthly_return_top_10_percent_weighted.append((date, weighted_monthly_return))

# Convert the list to a dataframe
monthly_return_top_10_percent_weighted_df = pd.DataFrame(monthly_return_top_10_percent_weighted, columns=["MthCalDt", "Weighted_Avg_Monthly_Return"])
